In [58]:
#pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [181]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time

In [183]:
import mysql.connector
from mysql.connector import Error

In [185]:
# Configuración del WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
driver = webdriver.Chrome(options=options)

In [186]:
# Conexión a la base de datos MySQL en Azure
try:
    connection = mysql.connector.connect(
        user="felipe",
        password="Felipe123",
        host="evaluacion-inacap-n3.mysql.database.azure.com",
        port=3306,
        database="evaluacionn3"
    )
    if connection.is_connected():
        print("Conexión exitosa a la base de datos")
except Error as e:
    print(f"Error al conectar a la base de datos: {e}")
    driver.quit()
    exit()

Conexión exitosa a la base de datos


#Comando para visualizar los datos de BD

USE evaluacionn3;

SHOW TABLES;

SELECT * FROM propiedades;

EXIT;

Implementar el siguiente codigo con la base de datos ya creada.

En mi caso cree una base de datos en azure llamada evaluacionn3.

# Crear un cursor para ejecutar comandos SQL
cursor = connection.cursor()

# SQL para crear la tabla
create_table_query = """
CREATE TABLE IF NOT EXISTS propiedades (
    id INT AUTO_INCREMENT PRIMARY KEY,
    precio VARCHAR(255),
    unidades_disponibles VARCHAR(50),
    superficie VARCHAR(50),
    dormitorios VARCHAR(50),
    banos VARCHAR(50),
    construccion VARCHAR(50),
    codigo VARCHAR(50),
    construye VARCHAR(255)
);
"""

# Ejecutar el comando para crear la tabla
try:
    cursor.execute(create_table_query)
    connection.commit()
    print("Tabla 'propiedades' creada exitosamente.")
except Exception as e:
    print(f"Error al crear la tabla: {e}")


In [192]:
# URL objetivo
url = 'https://www.portalinmobiliario.com/venta/casa/magallanes-y-antartica-chilena'
driver.get(url)

In [193]:
# Esperar a que los enlaces estén disponibles
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'ui-search-result__image.ui-search-link')))

[<selenium.webdriver.remote.webelement.WebElement (session="928b17953c5a523bb995dc1d22baa1fc", element="f.11386437898D53A99E70936A80368403.d.34A85E390174BD7663F50886A8303AA7.e.77")>,
 <selenium.webdriver.remote.webelement.WebElement (session="928b17953c5a523bb995dc1d22baa1fc", element="f.11386437898D53A99E70936A80368403.d.34A85E390174BD7663F50886A8303AA7.e.78")>,
 <selenium.webdriver.remote.webelement.WebElement (session="928b17953c5a523bb995dc1d22baa1fc", element="f.11386437898D53A99E70936A80368403.d.34A85E390174BD7663F50886A8303AA7.e.79")>,
 <selenium.webdriver.remote.webelement.WebElement (session="928b17953c5a523bb995dc1d22baa1fc", element="f.11386437898D53A99E70936A80368403.d.34A85E390174BD7663F50886A8303AA7.e.80")>,
 <selenium.webdriver.remote.webelement.WebElement (session="928b17953c5a523bb995dc1d22baa1fc", element="f.11386437898D53A99E70936A80368403.d.34A85E390174BD7663F50886A8303AA7.e.81")>,
 <selenium.webdriver.remote.webelement.WebElement (session="928b17953c5a523bb995dc1d2

In [196]:
# Set para almacenar los enlaces ya visitados
enlaces_visitados = set()

In [198]:
try:
    while True:
        # Encontrar todos los enlaces a propiedades en la página
        contenedores = driver.find_elements(By.CLASS_NAME, 'ui-search-result__image.ui-search-link')
        
        for contenedor in contenedores:
            try:
                # Obtener el enlace desde el contenedor
                enlace = contenedor.get_attribute('href')

                if enlace and enlace not in enlaces_visitados:
                    print(f"Navegando a: {enlace}")

                    # Agregar el enlace a los visitados
                    enlaces_visitados.add(enlace)

                    # Abrir una nueva ventana (pestaña)
                    driver.execute_script("window.open('');")
                    driver.switch_to.window(driver.window_handles[-1])
                    driver.get(enlace)  # Navegar a la página de la propiedad

                    # Esperar que la página cargue
                    time.sleep(5)

                    # Extraer la información de la propiedad
                    try:
                        precio = driver.find_element(By.CLASS_NAME, 'andes-money-amount__fraction').text
                    except Exception as e:
                        precio = "No disponible"
                        print(f"Error al obtener precio: {e}")

                    try:
                        unidades_disponibles = driver.find_element(
                            By.CLASS_NAME, 'ui-pdp-available-units-label'
                        ).text
                    except NoSuchElementException:
                        unidades_disponibles = None
                        print("Unidades disponibles no encontradas. Asignando NULL.")

                    try:
                        superficie = driver.find_element(By.XPATH, '//span[contains(text(), "m²")]').text
                    except Exception as e:
                        superficie = "No disponible"
                        print(f"Error al obtener superficie: {e}")

                    try:
                        dormitorios = driver.find_element(By.XPATH, '//span[contains(text(), "dormitorios")]').text
                    except Exception as e:
                        dormitorios = "No disponible"
                        print(f"Error al obtener dormitorios: {e}")

                    try:
                        baños = driver.find_element(By.XPATH, '//span[contains(text(), "baños")]').text
                    except NoSuchElementException:
                        try:
                            baños = driver.find_element(By.XPATH, '//span[contains(text(), "baño")]').text
                        except NoSuchElementException:
                            baños = "No disponible"
                            print("No se encontró 'baños' ni 'baño'.")

                    try:
                        construccion = driver.find_element(By.CLASS_NAME, 'ui-pdp-subtitle').text
                    except Exception as e:
                        construccion = "No disponible"
                        print(f"Error al obtener año de construcción: {e}")

                    # Nuevos datos: Código de la propiedad
                    try:
                        codigo = driver.find_element(By.XPATH, '//p[@class="ui-seller-info__status-info__subtitle"]').text
                    except Exception as e:
                        codigo = None  # Asignar NULL si no se encuentra
                        print(f"Error al obtener el código de la propiedad: {e}")

                    # Nuevos datos: Construye
                    try:
                        construye = driver.find_elements(By.XPATH, '//p[@class="ui-seller-info__status-info__subtitle"]')[1].text
                    except Exception as e:
                        construye = None  # Asignar NULL si no se encuentra
                        print(f"Error al obtener la constructora: {e}")

                    # Imprimir los datos en consola
                    print(f"Precio: {precio}")
                    print(f"Unidades disponibles: {unidades_disponibles}")
                    print(f"Superficie: {superficie}")
                    print(f"Dormitorios: {dormitorios}")
                    print(f"Baños: {baños}")
                    print(f"Año de construcción: {construccion}")
                    print(f"Código de propiedad: {codigo}")
                    print(f"Construye: {construye}")
                    print('-----------------------------------------')

                    # Insertar los datos en la base de datos
                    cursor = connection.cursor()
                    try:
                        query = """
                        INSERT INTO propiedades (precio, unidades_disponibles, superficie, dormitorios, banos, construccion, codigo, construye)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                        """
                        values = (precio, unidades_disponibles, superficie, dormitorios, baños, construccion, codigo, construye)
                        cursor.execute(query, values)
                        connection.commit()
                        print("Datos insertados en la base de datos.")
                    except Error as e:
                        print(f"Error al insertar en la base de datos: {e}")
                    finally:
                        cursor.close()

                    # Cerrar la ventana actual (de la propiedad)
                    driver.close()

                    # Volver a la ventana principal
                    driver.switch_to.window(driver.window_handles[0])

                    # Esperar a que la página principal cargue de nuevo
                    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'ui-search-result__image.ui-search-link')))
            
            except Exception as e:
                print(f"Error al procesar una propiedad: {e}")
        
        # Intentar hacer clic en la siguiente página (si está disponible)
        try:
            siguiente_pagina = driver.find_element(By.XPATH, '//a[@title="Siguiente"]')
            siguiente_pagina.click()
            wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'ui-search-result__image.ui-search-link')))
            print("Pasando a la siguiente página.")
        except Exception as e:
            print(f"Error al navegar a la siguiente página o no hay más páginas: {e}")
            break

except Exception as e:
    print(f"Error general: {e}")


Navegando a: https://www.portalinmobiliario.com/MLC-1457697319-alto-bulnes-_JM#position=1&search_layout=grid&type=item&tracking_id=333119c8-0f96-4f41-935d-ee2e22b4df58
Precio: 6.439
Unidades disponibles: 2 unidades disponibles
Superficie: 93.21 m² totales
Dormitorios: 3 dormitorios
Baños: 3 baños
Año de construcción: Proyecto - Entrega inmediata
Código de propiedad: 11204
Construye: Axis D.C.
-----------------------------------------
Datos insertados en la base de datos.
Navegando a: https://www.portalinmobiliario.com/MLC-1433147883-reserva-magallanes-_JM#position=2&search_layout=grid&type=item&tracking_id=333119c8-0f96-4f41-935d-ee2e22b4df58
Precio: 4.085
Unidades disponibles: 3 unidades disponibles
Superficie: 60 m² totales
Dormitorios: 3 dormitorios
Baños: 2 baños
Año de construcción: Proyecto - Entrega Inmediata
Código de propiedad: 11102
Construye: Inmobiliaria Socovesa Sur
-----------------------------------------
Datos insertados en la base de datos.
Navegando a: https://www.por

KeyboardInterrupt: 

In [159]:
# Crear un cursor para ejecutar comandos SQL
cursor = connection.cursor()

# Comando SQL para eliminar todos los registros
delete_query = "DELETE FROM propiedades;"
cursor.execute(delete_query)
connection.commit()

print("Todos los registros de la tabla 'propiedades' han sido eliminados.")

Todos los registros de la tabla 'propiedades' han sido eliminados.


In [157]:
# Comando SQL para eliminar la tabla
drop_table_query = "DROP TABLE IF EXISTS propiedades;"
cursor.execute(drop_table_query)
connection.commit()

ProgrammingError: Cursor is not connected

In [200]:
#cerrar conexión
if connection.is_connected():
    connection.close()
    print("Conexión cerrada")

Conexión cerrada
